
<a id='whatsnew-0130'></a>

# v0.13.0 (January 3, 2014)

{{ header }}

This is a major release from 0.12.0 and includes a number of API changes, several new features and
enhancements along with a large number of bug fixes.

Highlights include:

- support for a new index type `Float64Index`, and other Indexing enhancements  
- `HDFStore` has a new string based syntax for query specification  
- support for new methods of interpolation  
- updated `timedelta` operations  
- a new string manipulation method `extract`  
- Nanosecond support for Offsets  
- `isin` for DataFrames  


Several experimental features are added, including:

- new `eval/query` methods for expression evaluation  
- support for `msgpack` serialization  
- an i/o interface to Google’s `BigQuery`  


Their are several new or updated docs sections including:

- [Comparison with SQL](getting_started/comparison/comparison_with_sql.ipynb#compare-with-sql), which should be useful for those familiar with SQL but still learning pandas.  
- [Comparison with R](getting_started/comparison/comparison_with_r.ipynb#compare-with-r), idiom translations from R to pandas.  
- [Enhancing Performance](user_guide/enhancingperf.ipynb#enhancingperf), ways to enhance pandas performance with `eval/query`.  


In 0.13.0 `Series` has internally been refactored to no longer sub-class `ndarray`
but instead subclass `NDFrame`, similar to the rest of the pandas containers. This should be
a transparent change with only very limited API implications. See [Internal Refactoring](#whatsnew-0130-refactoring)

## API changes

- `read_excel` now supports an integer in its `sheetname` argument giving
  the index of the sheet to read in ([GH4301](https://github.com/pandas-dev/pandas/issues/4301)).  
- Text parser now treats anything that reads like inf (“inf”, “Inf”, “-Inf”,
  “iNf”, etc.) as infinity. ([GH4220](https://github.com/pandas-dev/pandas/issues/4220), [GH4219](https://github.com/pandas-dev/pandas/issues/4219)), affecting
  `read_table`, `read_csv`, etc.  
- `pandas` now is Python 2/3 compatible without the need for 2to3 thanks to
  @jtratner. As a result, pandas now uses iterators more extensively. This
  also led to the introduction of substantive parts of the Benjamin
  Peterson’s `six` library into compat. ([GH4384](https://github.com/pandas-dev/pandas/issues/4384), [GH4375](https://github.com/pandas-dev/pandas/issues/4375),
  [GH4372](https://github.com/pandas-dev/pandas/issues/4372))  
- `pandas.util.compat` and `pandas.util.py3compat` have been merged into
  `pandas.compat`. `pandas.compat` now includes many functions allowing
  2/3 compatibility. It contains both list and iterator versions of range,
  filter, map and zip, plus other necessary elements for Python 3
  compatibility. `lmap`, `lzip`, `lrange` and `lfilter` all produce
  lists instead of iterators, for compatibility with `numpy`, subscripting
  and `pandas` constructors.([GH4384](https://github.com/pandas-dev/pandas/issues/4384), [GH4375](https://github.com/pandas-dev/pandas/issues/4375), [GH4372](https://github.com/pandas-dev/pandas/issues/4372))  
- `Series.get` with negative indexers now returns the same as `[]` ([GH4390](https://github.com/pandas-dev/pandas/issues/4390))  
- Changes to how `Index` and `MultiIndex` handle metadata (`levels`,
`labels`, and `names`) ([GH4039](https://github.com/pandas-dev/pandas/issues/4039)):  

In [ ]:
# previously, you would have set levels or labels directly
>>> pd.index.levels = [[1, 2, 3, 4], [1, 2, 4, 4]]

# now, you use the set_levels or set_labels methods
>>> index = pd.index.set_levels([[1, 2, 3, 4], [1, 2, 4, 4]])

# similarly, for names, you can rename the object
# but setting names is not deprecated
>>> index = pd.index.set_names(["bob", "cranberry"])

# and all methods take an inplace kwarg - but return None
>>> pd.index.set_names(["bob", "cranberry"], inplace=True)

- **All** division with `NDFrame` objects is now *truedivision*, regardless
of the future import. This means that operating on pandas objects will by default
use *floating point* division, and return a floating point dtype.
You can use `//` and `floordiv` to do integer division.  
Integer division  

```ipython
In [3]: arr = np.array([1, 2, 3, 4])

In [4]: arr2 = np.array([5, 3, 2, 1])

In [5]: arr / arr2
Out[5]: array([0, 0, 1, 4])

In [6]: pd.Series(arr) // pd.Series(arr2)
Out[6]:
0    0
1    0
2    1
3    4
dtype: int64
```


True Division  

```ipython
In [7]: pd.Series(arr) / pd.Series(arr2)  # no future import required
Out[7]:
0    0.200000
1    0.666667
2    1.500000
3    4.000000
dtype: float64
```


- Infer and downcast dtype if `downcast='infer'` is passed to `fillna/ffill/bfill` ([GH4604](https://github.com/pandas-dev/pandas/issues/4604))  
- `__nonzero__` for all NDFrame objects, will now raise a `ValueError`, this reverts back to ([GH1073](https://github.com/pandas-dev/pandas/issues/1073), [GH4633](https://github.com/pandas-dev/pandas/issues/4633))
behavior. See [gotchas](user_guide/gotchas.ipynb#gotchas-truth) for a more detailed discussion.  
This prevents doing boolean comparison on *entire* pandas objects, which is inherently ambiguous. These all will raise a `ValueError`.  

In [ ]:
>>> df = pd.DataFrame({'A': np.random.randn(10),
...                    'B': np.random.randn(10),
...                    'C': pd.date_range('20130101', periods=10)
...                    })
...
>>> if df:
...     pass
...
Traceback (most recent call last):
    ...
ValueError: The truth value of a DataFrame is ambiguous.  Use a.empty,
a.bool(), a.item(), a.any() or a.all().

>>> df1 = df
>>> df2 = df
>>> df1 and df2
Traceback (most recent call last):
    ...
ValueError: The truth value of a DataFrame is ambiguous.  Use a.empty,
a.bool(), a.item(), a.any() or a.all().

>>> d = [1, 2, 3]
>>> s1 = pd.Series(d)
>>> s2 = pd.Series(d)
>>> s1 and s2
Traceback (most recent call last):
    ...
ValueError: The truth value of a DataFrame is ambiguous.  Use a.empty,
a.bool(), a.item(), a.any() or a.all().

Added the `.bool()` method to `NDFrame` objects to facilitate evaluating of single-element boolean Series:  
- All non-Index NDFrames (`Series`, `DataFrame`, `Panel`, `Panel4D`,
  `SparsePanel`, etc.), now support the entire set of arithmetic operators
  and arithmetic flex methods (add, sub, mul, etc.). `SparsePanel` does not
  support `pow` or `mod` with non-scalars. ([GH3765](https://github.com/pandas-dev/pandas/issues/3765))  
- `Series` and `DataFrame` now have a `mode()` method to calculate the
  statistical mode(s) by axis/Series. ([GH5367](https://github.com/pandas-dev/pandas/issues/5367))  
- Chained assignment will now by default warn if the user is assigning to a copy. This can be changed
with the option `mode.chained_assignment`, allowed options are `raise/warn/None`. See [the docs](user_guide/indexing.ipynb#indexing-view-versus-copy).  
The following warning / exception will show if this is attempted.  

In [ ]:
Traceback (most recent call last)
   ...
SettingWithCopyWarning:
   A value is trying to be set on a copy of a slice from a DataFrame.
   Try using .loc[row_index,col_indexer] = value instead

Here is the correct method of assignment.  
- 
  <dl style='margin: 20px 0;'>
  <dt>`Panel.reindex` has the following call signature `Panel.reindex(items=None, major_axis=None, minor_axis=None, **kwargs)`</dt>
  <dd>
  to conform with other `NDFrame` objects. See [Internal Refactoring](#whatsnew-0130-refactoring) for more information.  
  </dd>
  
  </dl>
  
- 
  <dl style='margin: 20px 0;'>
  <dt>`Series.argmin` and `Series.argmax` are now aliased to `Series.idxmin` and `Series.idxmax`. These return the *index* of the</dt>
  <dd>
  min or max element respectively. Prior to 0.13.0 these would return the position of the min / max element. ([GH6214](https://github.com/pandas-dev/pandas/issues/6214))  
  </dd>
  
  </dl>

## Prior version deprecations/changes

These were announced changes in 0.12 or prior that are taking effect as of 0.13.0

- Remove deprecated `Factor` ([GH3650](https://github.com/pandas-dev/pandas/issues/3650))  
- Remove deprecated `set_printoptions/reset_printoptions` ([GH3046](https://github.com/pandas-dev/pandas/issues/3046))  
- Remove deprecated `_verbose_info` ([GH3215](https://github.com/pandas-dev/pandas/issues/3215))  
- Remove deprecated `read_clipboard/to_clipboard/ExcelFile/ExcelWriter` from `pandas.io.parsers` ([GH3717](https://github.com/pandas-dev/pandas/issues/3717))
  These are available as functions in the main pandas namespace (e.g. `pd.read_clipboard`)  
- default for `tupleize_cols` is now `False` for both `to_csv` and `read_csv`. Fair warning in 0.12 ([GH3604](https://github.com/pandas-dev/pandas/issues/3604))  
- default for display.max_seq_len is now 100 rather then None. This activates
  truncated display (“…”) of long sequences in various places. ([GH3391](https://github.com/pandas-dev/pandas/issues/3391))  

## Deprecations

Deprecated in 0.13.0

- deprecated `iterkv`, which will be removed in a future release (this was
  an alias of iteritems used to bypass `2to3`’s changes).
  ([GH4384](https://github.com/pandas-dev/pandas/issues/4384), [GH4375](https://github.com/pandas-dev/pandas/issues/4375), [GH4372](https://github.com/pandas-dev/pandas/issues/4372))  
- deprecated the string method `match`, whose role is now performed more
  idiomatically by `extract`. In a future release, the default behavior
  of `match` will change to become analogous to `contains`, which returns
  a boolean indexer. (Their
  distinction is strictness: `match` relies on `re.match` while
  `contains` relies on `re.search`.) In this release, the deprecated
  behavior is the default, but the new behavior is available through the
  keyword argument `as_indexer=True`.  

## Indexing API changes

Prior to 0.13, it was impossible to use a label indexer (`.loc/.ix`) to set a value that
was not contained in the index of a particular axis. ([GH2578](https://github.com/pandas-dev/pandas/issues/2578)). See [the docs](user_guide/indexing.ipynb#indexing-basics-partial-setting)

In the `Series` case this is effectively an appending operation

This would previously `KeyError`

This is like an `append` operation.

A Panel setting operation on an arbitrary axis aligns the input to the Panel

```ipython
In [20]: p = pd.Panel(np.arange(16).reshape(2, 4, 2),
   ....:              items=['Item1', 'Item2'],
   ....:              major_axis=pd.date_range('2001/1/12', periods=4),
   ....:              minor_axis=['A', 'B'], dtype='float64')
   ....:

In [21]: p
Out[21]:
<class 'pandas.core.panel.Panel'>
Dimensions: 2 (items) x 4 (major_axis) x 2 (minor_axis)
Items axis: Item1 to Item2
Major_axis axis: 2001-01-12 00:00:00 to 2001-01-15 00:00:00
Minor_axis axis: A to B

In [22]: p.loc[:, :, 'C'] = pd.Series([30, 32], index=p.items)

In [23]: p
Out[23]:
<class 'pandas.core.panel.Panel'>
Dimensions: 2 (items) x 4 (major_axis) x 3 (minor_axis)
Items axis: Item1 to Item2
Major_axis axis: 2001-01-12 00:00:00 to 2001-01-15 00:00:00
Minor_axis axis: A to C

In [24]: p.loc[:, :, 'C']
Out[24]:
            Item1  Item2
2001-01-12   30.0   32.0
2001-01-13   30.0   32.0
2001-01-14   30.0   32.0
2001-01-15   30.0   32.0
```


## Float64Index API change

- Added a new index type, `Float64Index`. This will be automatically created when passing floating values in index creation.
  This enables a pure label-based slicing paradigm that makes `[],ix,loc` for scalar indexing and slicing work exactly the
  same. See [the docs](user_guide/advanced.ipynb#indexing-float64index), ([GH263](https://github.com/pandas-dev/pandas/issues/263))  
  Construction is by default for floating type values.  
  Scalar selection for `[],.ix,.loc` will always be label based. An integer will match an equal float index (e.g. `3` is equivalent to `3.0`)  
  The only positional indexing is via `iloc`  
  A scalar index that is not found will raise `KeyError`  
  Slicing is ALWAYS on the values of the index, for `[],ix,loc` and ALWAYS positional with `iloc`  
  In float indexes, slicing using floats are allowed  
- Indexing on other index types are preserved (and positional fallback for `[],ix`), with the exception, that floating point slicing
on indexes on non `Float64Index` will now raise a `TypeError`.  

```ipython
In [1]: pd.Series(range(5))[3.5]
TypeError: the label [3.5] is not a proper indexer for this index type (Int64Index)

In [1]: pd.Series(range(5))[3.5:4.5]
TypeError: the slice start [3.5] is not a proper indexer for this index type (Int64Index)
```


Using a scalar float indexer will be deprecated in a future version, but is allowed for now.  

```ipython
In [3]: pd.Series(range(5))[3.0]
Out[3]: 3
```




## HDFStore API changes

- Query Format Changes. A much more string-like query format is now supported. See [the docs](user_guide/io.ipynb#io-hdf5-query).  
  Use boolean expressions, with in-line function evaluation.  
  Use an inline column reference  
- the `format` keyword now replaces the `table` keyword; allowed values are `fixed(f)` or `table(t)`
  the same defaults as prior < 0.13.0 remain, e.g. `put` implies `fixed` format and `append` implies
  `table` format. This default format can be set as an option by setting `io.hdf.default_format`.  
- Significant table writing performance improvements  
- handle a passed `Series` in table format ([GH4330](https://github.com/pandas-dev/pandas/issues/4330))  
- can now serialize a `timedelta64[ns]` dtype in a table ([GH3577](https://github.com/pandas-dev/pandas/issues/3577)), See [the docs](user_guide/io.ipynb#io-hdf5-timedelta).  
- added an `is_open` property to indicate if the underlying file handle is_open;
  a closed store will now report ‘CLOSED’ when viewing the store (rather than raising an error)
  ([GH4409](https://github.com/pandas-dev/pandas/issues/4409))  
- a close of a `HDFStore` now will close that instance of the `HDFStore`
  but will only close the actual file if the ref count (by `PyTables`) w.r.t. all of the open handles
  are 0. Essentially you have a local instance of `HDFStore` referenced by a variable. Once you
  close it, it will report closed. Other references (to the same file) will continue to operate
  until they themselves are closed. Performing an action on a closed file will raise
  `ClosedFileError`  
- removed the `_quiet` attribute, replace by a `DuplicateWarning` if retrieving
  duplicate rows from a table ([GH4367](https://github.com/pandas-dev/pandas/issues/4367))  
- removed the `warn` argument from `open`. Instead a `PossibleDataLossError` exception will
  be raised if you try to use `mode='w'` with an OPEN file handle ([GH4367](https://github.com/pandas-dev/pandas/issues/4367))  
- allow a passed locations array or mask as a `where` condition ([GH4467](https://github.com/pandas-dev/pandas/issues/4467)).
  See [the docs](user_guide/io.ipynb#io-hdf5-where-mask) for an example.  
- add the keyword `dropna=True` to `append` to change whether ALL nan rows are not written
  to the store (default is `True`, ALL nan rows are NOT written), also settable
  via the option `io.hdf.dropna_table` ([GH4625](https://github.com/pandas-dev/pandas/issues/4625))  
- pass through store creation arguments; can be used to support in-memory stores  

## DataFrame repr changes

The HTML and plain text representations of `DataFrame` now show
a truncated view of the table once it exceeds a certain size, rather
than switching to the short info view ([GH4886](https://github.com/pandas-dev/pandas/issues/4886), [GH5550](https://github.com/pandas-dev/pandas/issues/5550)).
This makes the representation more consistent as small DataFrames get
larger.

<img src="whatsnew/../_static/df_repr_truncated.png" alt="Truncated HTML representation of a DataFrame" style="">

To get the info view, call `DataFrame.info()`. If you prefer the
info view as the repr for large DataFrames, you can set this by running
`set_option('display.large_repr', 'info')`.

## Enhancements

- `df.to_clipboard()` learned a new `excel` keyword that let’s you
  paste df data directly into excel (enabled by default). ([GH5070](https://github.com/pandas-dev/pandas/issues/5070)).  
- `read_html` now raises a `URLError` instead of catching and raising a
  `ValueError` ([GH4303](https://github.com/pandas-dev/pandas/issues/4303), [GH4305](https://github.com/pandas-dev/pandas/issues/4305))  
- Added a test for `read_clipboard()` and `to_clipboard()` ([GH4282](https://github.com/pandas-dev/pandas/issues/4282))  
- Clipboard functionality now works with PySide ([GH4282](https://github.com/pandas-dev/pandas/issues/4282))  
- Added a more informative error message when plot arguments contain
  overlapping color and style arguments ([GH4402](https://github.com/pandas-dev/pandas/issues/4402))  
- `to_dict` now takes `records` as a possible out type.  Returns an array
  of column-keyed dictionaries. ([GH4936](https://github.com/pandas-dev/pandas/issues/4936))  
- `NaN` handing in get_dummies ([GH4446](https://github.com/pandas-dev/pandas/issues/4446)) with dummy_na  
- `timedelta64[ns]` operations. See [the docs](user_guide/timedeltas.ipynb#timedeltas-timedeltas-convert).  
  Most of these operations require `numpy >= 1.7`  
  Using the new top-level `to_timedelta`, you can convert a scalar or array from the standard
  timedelta format (produced by `to_csv`) into a timedelta type (`np.timedelta64` in `nanoseconds`).  
  A Series of dtype `timedelta64[ns]` can now be divided by another
  `timedelta64[ns]` object, or astyped to yield a `float64` dtyped Series. This
  is frequency conversion. See [the docs](user_guide/timedeltas.ipynb#timedeltas-timedeltas-convert) for the docs.  
  Dividing or multiplying a `timedelta64[ns]` Series by an integer or integer Series  
  Absolute `DateOffset` objects can act equivalently to `timedeltas`  
  Fillna is now supported for timedeltas  
  You can do numeric reduction operations on timedeltas.  
- `plot(kind='kde')` now accepts the optional parameters `bw_method` and
  `ind`, passed to scipy.stats.gaussian_kde() (for scipy >= 0.11.0) to set
  the bandwidth, and to gkde.evaluate() to specify the indices at which it
  is evaluated, respectively. See scipy docs. ([GH4298](https://github.com/pandas-dev/pandas/issues/4298))  
- DataFrame constructor now accepts a numpy masked record array ([GH3478](https://github.com/pandas-dev/pandas/issues/3478))  
- The new vectorized string method `extract` return regular expression
  matches more conveniently.  
  Elements that do not match return `NaN`. Extracting a regular expression
  with more than one group returns a DataFrame with one column per group.  
  Elements that do not match return a row of `NaN`.
  Thus, a Series of messy strings can be *converted* into a
  like-indexed Series or DataFrame of cleaned-up or more useful strings,
  without necessitating `get()` to access tuples or `re.match` objects.  
  Named groups like  
  and optional groups can also be used.  
- `read_stata` now accepts Stata 13 format ([GH4291](https://github.com/pandas-dev/pandas/issues/4291))  
- `read_fwf` now infers the column specifications from the first 100 rows of
  the file if the data has correctly separated and properly aligned columns
  using the delimiter provided to the function ([GH4488](https://github.com/pandas-dev/pandas/issues/4488)).  
- support for nanosecond times as an offset  
  These operations require `numpy >= 1.7`  
  Period conversions in the range of seconds and below were reworked and extended
  up to nanoseconds. Periods in the nanosecond range are now available.  
  or with frequency as offset  
  Timestamps can be modified in the nanosecond range  
- A new method, `isin` for DataFrames, which plays nicely with boolean indexing. The argument to `isin`, what we’re comparing the DataFrame to, can be a DataFrame, Series, dict, or array of values. See [the docs](user_guide/indexing.ipynb#indexing-basics-indexing-isin) for more.  
  To get the rows where any of the conditions are met:  
- `Series` now supports a `to_frame` method to convert it to a single-column DataFrame ([GH5164](https://github.com/pandas-dev/pandas/issues/5164))  
- All R datasets listed here [http://stat.ethz.ch/R-manual/R-devel/library/datasets/html/00Index.html](http://stat.ethz.ch/R-manual/R-devel/library/datasets/html/00Index.html) can now be loaded into Pandas objects  

In [ ]:
# note that pandas.rpy was deprecated in v0.16.0
import pandas.rpy.common as com
com.load_data('Titanic')

- `tz_localize` can infer a fall daylight savings transition based on the structure
  of the unlocalized data ([GH4230](https://github.com/pandas-dev/pandas/issues/4230)), see [the docs](user_guide/timeseries.ipynb#timeseries-timezone)  
- `DatetimeIndex` is now in the API documentation, see the docs  
- `json_normalize()` is a new method to allow you to create a flat table
  from semi-structured JSON data. See [the docs](user_guide/io.ipynb#io-json-normalize) ([GH1067](https://github.com/pandas-dev/pandas/issues/1067))  
- Added PySide support for the qtpandas DataFrameModel and DataFrameWidget.  
- Python csv parser now supports usecols ([GH4335](https://github.com/pandas-dev/pandas/issues/4335))  
- Frequencies gained several new offsets:  
  - `LastWeekOfMonth` ([GH4637](https://github.com/pandas-dev/pandas/issues/4637))  
  - `FY5253`, and `FY5253Quarter` ([GH4511](https://github.com/pandas-dev/pandas/issues/4511))  
- DataFrame has a new `interpolate` method, similar to Series ([GH4434](https://github.com/pandas-dev/pandas/issues/4434), [GH1892](https://github.com/pandas-dev/pandas/issues/1892))  
  Additionally, the `method` argument to `interpolate` has been expanded
  to include `'nearest', 'zero', 'slinear', 'quadratic', 'cubic',
  'barycentric', 'krogh', 'piecewise_polynomial', 'pchip', 'polynomial', 'spline'`
  The new methods require [scipy](http://www.scipy.org). Consult the Scipy reference [guide](http://docs.scipy.org/doc/scipy/reference/tutorial/interpolate.html) and [documentation](http://docs.scipy.org/doc/scipy/reference/interpolate.html#univariate-interpolation) for more information
  about when the various methods are appropriate. See [the docs](user_guide/missing_data.ipynb#missing-data-interpolate).  
  Interpolate now also accepts a `limit` keyword argument.
  This works similar to `fillna`’s limit:  
- Added `wide_to_long` panel data convenience function. See [the docs](user_guide/reshaping.ipynb#reshaping-melt).  


- `to_csv` now takes a `date_format` keyword argument that specifies how
  output datetime objects should be formatted. Datetimes encountered in the
  index, columns, and values will all have this formatting applied. ([GH4313](https://github.com/pandas-dev/pandas/issues/4313))  
- `DataFrame.plot` will scatter plot x versus y by passing `kind='scatter'` ([GH2215](https://github.com/pandas-dev/pandas/issues/2215))  
- Added support for Google Analytics v3 API segment IDs that also supports v2 IDs. ([GH5271](https://github.com/pandas-dev/pandas/issues/5271))  



<a id='whatsnew-0130-experimental'></a>

## Experimental

- The new `eval()` function implements expression evaluation using
  `numexpr` behind the scenes. This results in large speedups for
  complicated expressions involving large DataFrames/Series. For example,  
  For more details, see the [the docs](user_guide/enhancingperf.ipynb#enhancingperf-eval)  
- Similar to `pandas.eval`, `DataFrame` has a new
  `DataFrame.eval` method that evaluates an expression in the context of
  the `DataFrame`. For example,  
- `query()` method has been added that allows
  you to select elements of a `DataFrame` using a natural query syntax
  nearly identical to Python syntax. For example,  
  selects all the rows of `df` where `a < b < c` evaluates to `True`.
  For more details see the [the docs](user_guide/indexing.ipynb#indexing-query).  
- `pd.read_msgpack()` and `pd.to_msgpack()` are now a supported method of serialization
  of arbitrary pandas (and python objects) in a lightweight portable binary format. See [the docs](user_guide/io.ipynb#io-msgpack)  
  Since this is an EXPERIMENTAL LIBRARY, the storage format may not be stable until a future release.  
  You can pass `iterator=True` to iterator over the unpacked results  
- `pandas.io.gbq` provides a simple way to extract from, and load data into,
Google’s BigQuery Data Sets by way of pandas DataFrames. BigQuery is a high
performance SQL-like database service, useful for performing ad-hoc queries
against extremely large datasets. [See the docs](user_guide/io.ipynb#io-bigquery)  

In [ ]:
from pandas.io import gbq

# A query to select the average monthly temperatures in the
# in the year 2000 across the USA. The dataset,
# publicata:samples.gsod, is available on all BigQuery accounts,
# and is based on NOAA gsod data.

query = """SELECT station_number as STATION,
month as MONTH, AVG(mean_temp) as MEAN_TEMP
FROM publicdata:samples.gsod
WHERE YEAR = 2000
GROUP BY STATION, MONTH
ORDER BY STATION, MONTH ASC"""

# Fetch the result set for this query

# Your Google BigQuery Project ID
# To find this, see your dashboard:
# https://console.developers.google.com/iam-admin/projects?authuser=0
projectid = 'xxxxxxxxx'
df = gbq.read_gbq(query, project_id=projectid)

# Use pandas to process and reshape the dataset

df2 = df.pivot(index='STATION', columns='MONTH', values='MEAN_TEMP')
df3 = pd.concat([df2.min(), df2.mean(), df2.max()],
                axis=1, keys=["Min Tem", "Mean Temp", "Max Temp"])

The resulting DataFrame is:  

In [ ]:
> df3
            Min Tem  Mean Temp    Max Temp
 MONTH
 1     -53.336667  39.827892   89.770968
 2     -49.837500  43.685219   93.437932
 3     -77.926087  48.708355   96.099998
 4     -82.892858  55.070087   97.317240
 5     -92.378261  61.428117  102.042856
 6     -77.703334  65.858888  102.900000
 7     -87.821428  68.169663  106.510714
 8     -89.431999  68.614215  105.500000
 9     -86.611112  63.436935  107.142856
 10    -78.209677  56.880838   92.103333
 11    -50.125000  48.861228   94.996428
 12    -50.332258  42.286879   94.396774

To use this module, you will need a BigQuery account. See
<[https://cloud.google.com/products/big-query](https://cloud.google.com/products/big-query)> for details.  
As of 10/10/13, there is a bug in Google’s API preventing result sets
from being larger than 100,000 rows. A patch is scheduled for the week of
10/14/13.  



<a id='whatsnew-0130-refactoring'></a>

## Internal refactoring

In 0.13.0 there is a major refactor primarily to subclass `Series` from
`NDFrame`, which is the base class currently for `DataFrame` and `Panel`,
to unify methods and behaviors. Series formerly subclassed directly from
`ndarray`. ([GH4080](https://github.com/pandas-dev/pandas/issues/4080), [GH3862](https://github.com/pandas-dev/pandas/issues/3862), [GH816](https://github.com/pandas-dev/pandas/issues/816))

There are two potential incompatibilities from < 0.13.0

- Using certain numpy functions would previously return a `Series` if passed a `Series`
  as an argument. This seems only to affect `np.ones_like`, `np.empty_like`,
  `np.diff` and `np.where`. These now return `ndarrays`.  
  Numpy Usage  
  Pandonic Usage  
- Passing a `Series` directly to a cython function expecting an `ndarray` type will no
  long work directly, you must pass `Series.values`, See [Enhancing Performance](user_guide/enhancingperf.ipynb#enhancingperf-ndarray)  
- `Series(0.5)` would previously return the scalar `0.5`, instead this will return a 1-element `Series`  
- This change breaks `rpy2<=2.3.8`. an Issue has been opened against rpy2 and a workaround
  is detailed in [GH5698](https://github.com/pandas-dev/pandas/issues/5698). Thanks @JanSchulz.  


- Pickle compatibility is preserved for pickles created prior to 0.13. These must be unpickled with `pd.read_pickle`, see [Pickling](user_guide/io.ipynb#io-pickle).  
- Refactor of series.py/frame.py/panel.py to move common code to generic.py  
  - added `_setup_axes` to created generic NDFrame structures  
  - moved methods  
    - `from_axes,_wrap_array,axes,ix,loc,iloc,shape,empty,swapaxes,transpose,pop`  
    - `__iter__,keys,__contains__,__len__,__neg__,__invert__`  
    - `convert_objects,as_blocks,as_matrix,values`  
    - `__getstate__,__setstate__` (compat remains in frame/panel)  
    - `__getattr__,__setattr__`  
    - `_indexed_same,reindex_like,align,where,mask`  
    - `fillna,replace` (`Series` replace is now consistent with `DataFrame`)  
    - `filter` (also added axis argument to selectively filter on a different axis)  
    - `reindex,reindex_axis,take`  
    - `truncate` (moved to become part of `NDFrame`)  
- These are API changes which make `Panel` more consistent with `DataFrame`  
  - `swapaxes` on a `Panel` with the same axes specified now return a copy  
  - support attribute access for setting  
  - filter supports the same API as the original `DataFrame` filter  
- Reindex called with no arguments will now return a copy of the input object  
- `TimeSeries` is now an alias for `Series`. the property `is_time_series`
  can be used to distinguish (if desired)  
- Refactor of Sparse objects to use BlockManager  
  - Created a new block type in internals, `SparseBlock`, which can hold multi-dtypes
    and is non-consolidatable. `SparseSeries` and `SparseDataFrame` now inherit
    more methods from there hierarchy (Series/DataFrame), and no longer inherit
    from `SparseArray` (which instead is the object of the `SparseBlock`)  
  - Sparse suite now supports integration with non-sparse data. Non-float sparse
    data is supportable (partially implemented)  
  - Operations on sparse structures within DataFrames should preserve sparseness,
    merging type operations will convert to dense (and back to sparse), so might
    be somewhat inefficient  
  - enable setitem on `SparseSeries` for boolean/integer/slices  
  - `SparsePanels` implementation is unchanged (e.g. not using BlockManager, needs work)  
- added `ftypes` method to Series/DataFrame, similar to `dtypes`, but indicates
  if the underlying is sparse/dense (as well as the dtype)  
- All `NDFrame` objects can now use `__finalize__()` to specify various
  values to propagate to new objects from an existing one (e.g. `name` in `Series` will
  follow more automatically now)  
- Internal type checking is now done via a suite of generated classes, allowing `isinstance(value, klass)`
  without having to directly import the klass, courtesy of @jtratner  
- Bug in Series update where the parent frame is not updating its cache based on
  changes ([GH4080](https://github.com/pandas-dev/pandas/issues/4080)) or types ([GH3217](https://github.com/pandas-dev/pandas/issues/3217)), fillna ([GH3386](https://github.com/pandas-dev/pandas/issues/3386))  
- Indexing with dtype conversions fixed ([GH4463](https://github.com/pandas-dev/pandas/issues/4463), [GH4204](https://github.com/pandas-dev/pandas/issues/4204))  
- Refactor `Series.reindex` to core/generic.py ([GH4604](https://github.com/pandas-dev/pandas/issues/4604), [GH4618](https://github.com/pandas-dev/pandas/issues/4618)), allow `method=` in reindexing
  on a Series to work  
- `Series.copy` no longer accepts the `order` parameter and is now consistent with `NDFrame` copy  
- Refactor `rename` methods to core/generic.py; fixes `Series.rename` for ([GH4605](https://github.com/pandas-dev/pandas/issues/4605)), and adds `rename`
  with the same signature for `Panel`  
- Refactor `clip` methods to core/generic.py ([GH4798](https://github.com/pandas-dev/pandas/issues/4798))  
- Refactor of `_get_numeric_data/_get_bool_data` to core/generic.py, allowing Series/Panel functionality  
- `Series` (for index) / `Panel` (for items) now allow attribute access to its elements  ([GH1903](https://github.com/pandas-dev/pandas/issues/1903))  



<a id='release-bug-fixes-0-13-0'></a>

## Bug fixes

- `HDFStore`  
  - raising an invalid `TypeError` rather than `ValueError` when
    appending with a different block ordering ([GH4096](https://github.com/pandas-dev/pandas/issues/4096))  
  - `read_hdf` was not respecting as passed `mode` ([GH4504](https://github.com/pandas-dev/pandas/issues/4504))  
  - appending a 0-len table will work correctly ([GH4273](https://github.com/pandas-dev/pandas/issues/4273))  
  - `to_hdf` was raising when passing both arguments `append` and
    `table` ([GH4584](https://github.com/pandas-dev/pandas/issues/4584))  
  - reading from a store with duplicate columns across dtypes would raise
    ([GH4767](https://github.com/pandas-dev/pandas/issues/4767))  
  - Fixed a bug where `ValueError` wasn’t correctly raised when column
    names weren’t strings ([GH4956](https://github.com/pandas-dev/pandas/issues/4956))  
  - A zero length series written in Fixed format not deserializing properly.
    ([GH4708](https://github.com/pandas-dev/pandas/issues/4708))  
  - Fixed decoding perf issue on pyt3 ([GH5441](https://github.com/pandas-dev/pandas/issues/5441))  
  - Validate levels in a MultiIndex before storing ([GH5527](https://github.com/pandas-dev/pandas/issues/5527))  
  - Correctly handle `data_columns` with a Panel ([GH5717](https://github.com/pandas-dev/pandas/issues/5717))  
- Fixed bug in tslib.tz_convert(vals, tz1, tz2): it could raise IndexError
  exception while trying to access trans[pos + 1] ([GH4496](https://github.com/pandas-dev/pandas/issues/4496))  
- The `by` argument now works correctly with the `layout` argument
  ([GH4102](https://github.com/pandas-dev/pandas/issues/4102), [GH4014](https://github.com/pandas-dev/pandas/issues/4014)) in `*.hist` plotting methods  
- Fixed bug in `PeriodIndex.map` where using `str` would return the str
  representation of the index ([GH4136](https://github.com/pandas-dev/pandas/issues/4136))  
- Fixed test failure `test_time_series_plot_color_with_empty_kwargs` when
  using custom matplotlib default colors ([GH4345](https://github.com/pandas-dev/pandas/issues/4345))  
- Fix running of stata IO tests. Now uses temporary files to write
  ([GH4353](https://github.com/pandas-dev/pandas/issues/4353))  
- Fixed an issue where `DataFrame.sum` was slower than `DataFrame.mean`
  for integer valued frames ([GH4365](https://github.com/pandas-dev/pandas/issues/4365))  
- `read_html` tests now work with Python 2.6 ([GH4351](https://github.com/pandas-dev/pandas/issues/4351))  
- Fixed bug where `network` testing was throwing `NameError` because a
  local variable was undefined ([GH4381](https://github.com/pandas-dev/pandas/issues/4381))  
- In `to_json`, raise if a passed `orient` would cause loss of data
  because of a duplicate index ([GH4359](https://github.com/pandas-dev/pandas/issues/4359))  
- In `to_json`, fix date handling so milliseconds are the default timestamp
  as the docstring says ([GH4362](https://github.com/pandas-dev/pandas/issues/4362)).  
- `as_index` is no longer ignored when doing groupby apply ([GH4648](https://github.com/pandas-dev/pandas/issues/4648),
  [GH3417](https://github.com/pandas-dev/pandas/issues/3417))  
- JSON NaT handling fixed, NaTs are now serialized to null ([GH4498](https://github.com/pandas-dev/pandas/issues/4498))  
- Fixed JSON handling of escapable characters in JSON object keys
  ([GH4593](https://github.com/pandas-dev/pandas/issues/4593))  
- Fixed passing `keep_default_na=False` when `na_values=None`
  ([GH4318](https://github.com/pandas-dev/pandas/issues/4318))  
- Fixed bug with `values` raising an error on a DataFrame with duplicate
  columns and mixed dtypes, surfaced in ([GH4377](https://github.com/pandas-dev/pandas/issues/4377))  
- Fixed bug with duplicate columns and type conversion in `read_json` when
  `orient='split'` ([GH4377](https://github.com/pandas-dev/pandas/issues/4377))  
- Fixed JSON bug where locales with decimal separators other than ‘.’ threw
  exceptions when encoding / decoding certain values. ([GH4918](https://github.com/pandas-dev/pandas/issues/4918))  
- Fix `.iat` indexing with a `PeriodIndex` ([GH4390](https://github.com/pandas-dev/pandas/issues/4390))  
- Fixed an issue where `PeriodIndex` joining with self was returning a new
  instance rather than the same instance ([GH4379](https://github.com/pandas-dev/pandas/issues/4379)); also adds a test
  for this for the other index types  
- Fixed a bug with all the dtypes being converted to object when using the
  CSV cparser with the usecols parameter ([GH3192](https://github.com/pandas-dev/pandas/issues/3192))  
- Fix an issue in merging blocks where the resulting DataFrame had partially
  set _ref_locs ([GH4403](https://github.com/pandas-dev/pandas/issues/4403))  
- Fixed an issue where hist subplots were being overwritten when they were
  called using the top level matplotlib API ([GH4408](https://github.com/pandas-dev/pandas/issues/4408))  
- Fixed a bug where calling `Series.astype(str)` would truncate the string
  ([GH4405](https://github.com/pandas-dev/pandas/issues/4405), [GH4437](https://github.com/pandas-dev/pandas/issues/4437))  
- Fixed a py3 compat issue where bytes were being repr’d as tuples
  ([GH4455](https://github.com/pandas-dev/pandas/issues/4455))  
- Fixed Panel attribute naming conflict if item is named ‘a’
  ([GH3440](https://github.com/pandas-dev/pandas/issues/3440))  
- Fixed an issue where duplicate indexes were raising when plotting
  ([GH4486](https://github.com/pandas-dev/pandas/issues/4486))  
- Fixed an issue where cumsum and cumprod didn’t work with bool dtypes
  ([GH4170](https://github.com/pandas-dev/pandas/issues/4170), [GH4440](https://github.com/pandas-dev/pandas/issues/4440))  
- Fixed Panel slicing issued in `xs` that was returning an incorrect dimmed
  object ([GH4016](https://github.com/pandas-dev/pandas/issues/4016))  
- Fix resampling bug where custom reduce function not used if only one group
  ([GH3849](https://github.com/pandas-dev/pandas/issues/3849), [GH4494](https://github.com/pandas-dev/pandas/issues/4494))  
- Fixed Panel assignment with a transposed frame ([GH3830](https://github.com/pandas-dev/pandas/issues/3830))  
- Raise on set indexing with a Panel and a Panel as a value which needs
  alignment ([GH3777](https://github.com/pandas-dev/pandas/issues/3777))  
- frozenset objects now raise in the `Series` constructor ([GH4482](https://github.com/pandas-dev/pandas/issues/4482),
  [GH4480](https://github.com/pandas-dev/pandas/issues/4480))  
- Fixed issue with sorting a duplicate MultiIndex that has multiple dtypes
  ([GH4516](https://github.com/pandas-dev/pandas/issues/4516))  
- Fixed bug in `DataFrame.set_values` which was causing name attributes to
  be lost when expanding the index. ([GH3742](https://github.com/pandas-dev/pandas/issues/3742), [GH4039](https://github.com/pandas-dev/pandas/issues/4039))  
- Fixed issue where individual `names`, `levels` and `labels` could be
  set on `MultiIndex` without validation ([GH3714](https://github.com/pandas-dev/pandas/issues/3714), [GH4039](https://github.com/pandas-dev/pandas/issues/4039))  
- Fixed ([GH3334](https://github.com/pandas-dev/pandas/issues/3334)) in pivot_table. Margins did not compute if values is
  the index.  
- Fix bug in having a rhs of `np.timedelta64` or `np.offsets.DateOffset`
  when operating with datetimes ([GH4532](https://github.com/pandas-dev/pandas/issues/4532))  
- Fix arithmetic with series/datetimeindex and `np.timedelta64` not working
  the same ([GH4134](https://github.com/pandas-dev/pandas/issues/4134)) and buggy timedelta in NumPy 1.6 ([GH4135](https://github.com/pandas-dev/pandas/issues/4135))  
- Fix bug in `pd.read_clipboard` on windows with PY3 ([GH4561](https://github.com/pandas-dev/pandas/issues/4561)); not
  decoding properly  
- `tslib.get_period_field()` and `tslib.get_period_field_arr()` now raise
  if code argument out of range ([GH4519](https://github.com/pandas-dev/pandas/issues/4519), [GH4520](https://github.com/pandas-dev/pandas/issues/4520))  
- Fix boolean indexing on an empty series loses index names ([GH4235](https://github.com/pandas-dev/pandas/issues/4235)),
  infer_dtype works with empty arrays.  
- Fix reindexing with multiple axes; if an axes match was not replacing the
  current axes, leading to a possible lazy frequency inference issue
  ([GH3317](https://github.com/pandas-dev/pandas/issues/3317))  
- Fixed issue where `DataFrame.apply` was reraising exceptions incorrectly
  (causing the original stack trace to be truncated).  
- Fix selection with `ix/loc` and non_unique selectors ([GH4619](https://github.com/pandas-dev/pandas/issues/4619))  
- Fix assignment with iloc/loc involving a dtype change in an existing column
  ([GH4312](https://github.com/pandas-dev/pandas/issues/4312), [GH5702](https://github.com/pandas-dev/pandas/issues/5702)) have internal setitem_with_indexer in core/indexing
  to use Block.setitem  
- Fixed bug where thousands operator was not handled correctly for floating
  point numbers in csv_import ([GH4322](https://github.com/pandas-dev/pandas/issues/4322))  
- Fix an issue with CacheableOffset not properly being used by many
  DateOffset; this prevented the DateOffset from being cached ([GH4609](https://github.com/pandas-dev/pandas/issues/4609))  
- Fix boolean comparison with a DataFrame on the lhs, and a list/tuple on the
  rhs ([GH4576](https://github.com/pandas-dev/pandas/issues/4576))  
- Fix error/dtype conversion with setitem of `None` on `Series/DataFrame`
  ([GH4667](https://github.com/pandas-dev/pandas/issues/4667))  
- Fix decoding based on a passed in non-default encoding in `pd.read_stata`
  ([GH4626](https://github.com/pandas-dev/pandas/issues/4626))  
- Fix `DataFrame.from_records` with a plain-vanilla `ndarray`.
  ([GH4727](https://github.com/pandas-dev/pandas/issues/4727))  
- Fix some inconsistencies with `Index.rename` and `MultiIndex.rename`,
  etc. ([GH4718](https://github.com/pandas-dev/pandas/issues/4718), [GH4628](https://github.com/pandas-dev/pandas/issues/4628))  
- Bug in using `iloc/loc` with a cross-sectional and duplicate indices
  ([GH4726](https://github.com/pandas-dev/pandas/issues/4726))  
- Bug with using `QUOTE_NONE` with `to_csv` causing `Exception`.
  ([GH4328](https://github.com/pandas-dev/pandas/issues/4328))  
- Bug with Series indexing not raising an error when the right-hand-side has
  an incorrect length ([GH2702](https://github.com/pandas-dev/pandas/issues/2702))  
- Bug in MultiIndexing with a partial string selection as one part of a
  MultIndex ([GH4758](https://github.com/pandas-dev/pandas/issues/4758))  
- Bug with reindexing on the index with a non-unique index will now raise
  `ValueError` ([GH4746](https://github.com/pandas-dev/pandas/issues/4746))  
- Bug in setting with `loc/ix` a single indexer with a MultiIndex axis and
  a NumPy array, related to ([GH3777](https://github.com/pandas-dev/pandas/issues/3777))  
- Bug in concatenation with duplicate columns across dtypes not merging with
  axis=0 ([GH4771](https://github.com/pandas-dev/pandas/issues/4771), [GH4975](https://github.com/pandas-dev/pandas/issues/4975))  
- Bug in `iloc` with a slice index failing ([GH4771](https://github.com/pandas-dev/pandas/issues/4771))  
- Incorrect error message with no colspecs or width in `read_fwf`.
  ([GH4774](https://github.com/pandas-dev/pandas/issues/4774))  
- Fix bugs in indexing in a Series with a duplicate index ([GH4548](https://github.com/pandas-dev/pandas/issues/4548),
  [GH4550](https://github.com/pandas-dev/pandas/issues/4550))  
- Fixed bug with reading compressed files with `read_fwf` in Python 3.
  ([GH3963](https://github.com/pandas-dev/pandas/issues/3963))  
- Fixed an issue with a duplicate index and assignment with a dtype change
  ([GH4686](https://github.com/pandas-dev/pandas/issues/4686))  
- Fixed bug with reading compressed files in as `bytes` rather than `str`
  in Python 3. Simplifies bytes-producing file-handling in Python 3
  ([GH3963](https://github.com/pandas-dev/pandas/issues/3963), [GH4785](https://github.com/pandas-dev/pandas/issues/4785)).  
- Fixed an issue related to ticklocs/ticklabels with log scale bar plots
  across different versions of matplotlib ([GH4789](https://github.com/pandas-dev/pandas/issues/4789))  
- Suppressed DeprecationWarning associated with internal calls issued by
  repr() ([GH4391](https://github.com/pandas-dev/pandas/issues/4391))  
- Fixed an issue with a duplicate index and duplicate selector with `.loc`
  ([GH4825](https://github.com/pandas-dev/pandas/issues/4825))  
- Fixed an issue with `DataFrame.sort_index` where, when sorting by a
  single column and passing a list for `ascending`, the argument for
  `ascending` was being interpreted as `True` ([GH4839](https://github.com/pandas-dev/pandas/issues/4839),
  [GH4846](https://github.com/pandas-dev/pandas/issues/4846))  
- Fixed `Panel.tshift` not working. Added freq support to `Panel.shift`
  ([GH4853](https://github.com/pandas-dev/pandas/issues/4853))  
- Fix an issue in TextFileReader w/ Python engine (i.e. PythonParser)
  with thousands != “,” ([GH4596](https://github.com/pandas-dev/pandas/issues/4596))  
- Bug in getitem with a duplicate index when using where ([GH4879](https://github.com/pandas-dev/pandas/issues/4879))  
- Fix Type inference code coerces float column into datetime ([GH4601](https://github.com/pandas-dev/pandas/issues/4601))  
- Fixed `_ensure_numeric` does not check for complex numbers
  ([GH4902](https://github.com/pandas-dev/pandas/issues/4902))  
- Fixed a bug in `Series.hist` where two figures were being created when
  the `by` argument was passed ([GH4112](https://github.com/pandas-dev/pandas/issues/4112), [GH4113](https://github.com/pandas-dev/pandas/issues/4113)).  
- Fixed a bug in `convert_objects` for > 2 ndims ([GH4937](https://github.com/pandas-dev/pandas/issues/4937))  
- Fixed a bug in DataFrame/Panel cache insertion and subsequent indexing
  ([GH4939](https://github.com/pandas-dev/pandas/issues/4939), [GH5424](https://github.com/pandas-dev/pandas/issues/5424))  
- Fixed string methods for `FrozenNDArray` and `FrozenList`
  ([GH4929](https://github.com/pandas-dev/pandas/issues/4929))  
- Fixed a bug with setting invalid or out-of-range values in indexing
  enlargement scenarios ([GH4940](https://github.com/pandas-dev/pandas/issues/4940))  
- Tests for fillna on empty Series ([GH4346](https://github.com/pandas-dev/pandas/issues/4346)), thanks @immerrr  
- Fixed `copy()` to shallow copy axes/indices as well and thereby keep
  separate metadata. ([GH4202](https://github.com/pandas-dev/pandas/issues/4202), [GH4830](https://github.com/pandas-dev/pandas/issues/4830))  
- Fixed skiprows option in Python parser for read_csv ([GH4382](https://github.com/pandas-dev/pandas/issues/4382))  
- Fixed bug preventing `cut` from working with `np.inf` levels without
  explicitly passing labels ([GH3415](https://github.com/pandas-dev/pandas/issues/3415))  
- Fixed wrong check for overlapping in `DatetimeIndex.union`
  ([GH4564](https://github.com/pandas-dev/pandas/issues/4564))  
- Fixed conflict between thousands separator and date parser in csv_parser
  ([GH4678](https://github.com/pandas-dev/pandas/issues/4678))  
- Fix appending when dtypes are not the same (error showing mixing
  float/np.datetime64) ([GH4993](https://github.com/pandas-dev/pandas/issues/4993))  
- Fix repr for DateOffset. No longer show duplicate entries in kwds.
  Removed unused offset fields. ([GH4638](https://github.com/pandas-dev/pandas/issues/4638))  
- Fixed wrong index name during read_csv if using usecols. Applies to c
  parser only. ([GH4201](https://github.com/pandas-dev/pandas/issues/4201))  
- `Timestamp` objects can now appear in the left hand side of a comparison
  operation with a `Series` or `DataFrame` object ([GH4982](https://github.com/pandas-dev/pandas/issues/4982)).  
- Fix a bug when indexing with `np.nan` via `iloc/loc` ([GH5016](https://github.com/pandas-dev/pandas/issues/5016))  
- Fixed a bug where low memory c parser could create different types in
  different chunks of the same file. Now coerces to numerical type or raises
  warning. ([GH3866](https://github.com/pandas-dev/pandas/issues/3866))  
- Fix a bug where reshaping a `Series` to its own shape raised
  `TypeError` ([GH4554](https://github.com/pandas-dev/pandas/issues/4554)) and other reshaping issues.  
- Bug in setting with `ix/loc` and a mixed int/string index ([GH4544](https://github.com/pandas-dev/pandas/issues/4544))  
- Make sure series-series boolean comparisons are label based ([GH4947](https://github.com/pandas-dev/pandas/issues/4947))  
- Bug in multi-level indexing with a Timestamp partial indexer
  ([GH4294](https://github.com/pandas-dev/pandas/issues/4294))  
- Tests/fix for MultiIndex construction of an all-nan frame ([GH4078](https://github.com/pandas-dev/pandas/issues/4078))  
- Fixed a bug where `read_html()` wasn’t correctly inferring
  values of tables with commas ([GH5029](https://github.com/pandas-dev/pandas/issues/5029))  
- Fixed a bug where `read_html()` wasn’t providing a stable
  ordering of returned tables ([GH4770](https://github.com/pandas-dev/pandas/issues/4770), [GH5029](https://github.com/pandas-dev/pandas/issues/5029)).  
- Fixed a bug where `read_html()` was incorrectly parsing when
  passed `index_col=0` ([GH5066](https://github.com/pandas-dev/pandas/issues/5066)).  
- Fixed a bug where `read_html()` was incorrectly inferring the
  type of headers ([GH5048](https://github.com/pandas-dev/pandas/issues/5048)).  
- Fixed a bug where `DatetimeIndex` joins with `PeriodIndex` caused a
  stack overflow ([GH3899](https://github.com/pandas-dev/pandas/issues/3899)).  
- Fixed a bug where `groupby` objects didn’t allow plots ([GH5102](https://github.com/pandas-dev/pandas/issues/5102)).  
- Fixed a bug where `groupby` objects weren’t tab-completing column names
  ([GH5102](https://github.com/pandas-dev/pandas/issues/5102)).  
- Fixed a bug where `groupby.plot()` and friends were duplicating figures
  multiple times ([GH5102](https://github.com/pandas-dev/pandas/issues/5102)).  
- Provide automatic conversion of `object` dtypes on fillna, related
  ([GH5103](https://github.com/pandas-dev/pandas/issues/5103))  
- Fixed a bug where default options were being overwritten in the option
  parser cleaning ([GH5121](https://github.com/pandas-dev/pandas/issues/5121)).  
- Treat a list/ndarray identically for `iloc` indexing with list-like
  ([GH5006](https://github.com/pandas-dev/pandas/issues/5006))  
- Fix `MultiIndex.get_level_values()` with missing values ([GH5074](https://github.com/pandas-dev/pandas/issues/5074))  
- Fix bound checking for Timestamp() with datetime64 input ([GH4065](https://github.com/pandas-dev/pandas/issues/4065))  
- Fix a bug where `TestReadHtml` wasn’t calling the correct `read_html()`
  function ([GH5150](https://github.com/pandas-dev/pandas/issues/5150)).  
- Fix a bug with `NDFrame.replace()` which made replacement appear as
  though it was (incorrectly) using regular expressions ([GH5143](https://github.com/pandas-dev/pandas/issues/5143)).  
- Fix better error message for to_datetime ([GH4928](https://github.com/pandas-dev/pandas/issues/4928))  
- Made sure different locales are tested on travis-ci ([GH4918](https://github.com/pandas-dev/pandas/issues/4918)). Also
  adds a couple of utilities for getting locales and setting locales with a
  context manager.  
- Fixed segfault on `isnull(MultiIndex)` (now raises an error instead)
  ([GH5123](https://github.com/pandas-dev/pandas/issues/5123), [GH5125](https://github.com/pandas-dev/pandas/issues/5125))  
- Allow duplicate indices when performing operations that align
  ([GH5185](https://github.com/pandas-dev/pandas/issues/5185), [GH5639](https://github.com/pandas-dev/pandas/issues/5639))  
- Compound dtypes in a constructor raise `NotImplementedError`
  ([GH5191](https://github.com/pandas-dev/pandas/issues/5191))  
- Bug in comparing duplicate frames ([GH4421](https://github.com/pandas-dev/pandas/issues/4421)) related  
- Bug in describe on duplicate frames  
- Bug in `to_datetime` with a format and `coerce=True` not raising
  ([GH5195](https://github.com/pandas-dev/pandas/issues/5195))  
- Bug in `loc` setting with multiple indexers and a rhs of a Series that
  needs broadcasting ([GH5206](https://github.com/pandas-dev/pandas/issues/5206))  
- Fixed bug where inplace setting of levels or labels on `MultiIndex` would
  not clear cached `values` property and therefore return wrong `values`.
  ([GH5215](https://github.com/pandas-dev/pandas/issues/5215))  
- Fixed bug where filtering a grouped DataFrame or Series did not maintain
  the original ordering ([GH4621](https://github.com/pandas-dev/pandas/issues/4621)).  
- Fixed `Period` with a business date freq to always roll-forward if on a
  non-business date. ([GH5203](https://github.com/pandas-dev/pandas/issues/5203))  
- Fixed bug in Excel writers where frames with duplicate column names weren’t
  written correctly. ([GH5235](https://github.com/pandas-dev/pandas/issues/5235))  
- Fixed issue with `drop` and a non-unique index on Series ([GH5248](https://github.com/pandas-dev/pandas/issues/5248))  
- Fixed segfault in C parser caused by passing more names than columns in
  the file. ([GH5156](https://github.com/pandas-dev/pandas/issues/5156))  
- Fix `Series.isin` with date/time-like dtypes ([GH5021](https://github.com/pandas-dev/pandas/issues/5021))  
- C and Python Parser can now handle the more common MultiIndex column
  format which doesn’t have a row for index names ([GH4702](https://github.com/pandas-dev/pandas/issues/4702))  
- Bug when trying to use an out-of-bounds date as an object dtype
  ([GH5312](https://github.com/pandas-dev/pandas/issues/5312))  
- Bug when trying to display an embedded PandasObject ([GH5324](https://github.com/pandas-dev/pandas/issues/5324))  
- Allows operating of Timestamps to return a datetime if the result is out-of-bounds
  related ([GH5312](https://github.com/pandas-dev/pandas/issues/5312))  
- Fix return value/type signature of `initObjToJSON()` to be compatible
  with numpy’s `import_array()` ([GH5334](https://github.com/pandas-dev/pandas/issues/5334), [GH5326](https://github.com/pandas-dev/pandas/issues/5326))  
- Bug when renaming then set_index on a DataFrame ([GH5344](https://github.com/pandas-dev/pandas/issues/5344))  
- Test suite no longer leaves around temporary files when testing graphics. ([GH5347](https://github.com/pandas-dev/pandas/issues/5347))
  (thanks for catching this @yarikoptic!)  
- Fixed html tests on win32. ([GH4580](https://github.com/pandas-dev/pandas/issues/4580))  
- Make sure that `head/tail` are `iloc` based, ([GH5370](https://github.com/pandas-dev/pandas/issues/5370))  
- Fixed bug for `PeriodIndex` string representation if there are 1 or 2
  elements. ([GH5372](https://github.com/pandas-dev/pandas/issues/5372))  
- The GroupBy methods `transform` and `filter` can be used on Series
  and DataFrames that have repeated (non-unique) indices. ([GH4620](https://github.com/pandas-dev/pandas/issues/4620))  
- Fix empty series not printing name in repr ([GH4651](https://github.com/pandas-dev/pandas/issues/4651))  
- Make tests create temp files in temp directory by default. ([GH5419](https://github.com/pandas-dev/pandas/issues/5419))  
- `pd.to_timedelta` of a scalar returns a scalar ([GH5410](https://github.com/pandas-dev/pandas/issues/5410))  
- `pd.to_timedelta` accepts `NaN` and `NaT`, returning `NaT` instead of raising ([GH5437](https://github.com/pandas-dev/pandas/issues/5437))  
- performance improvements in `isnull` on larger size pandas objects  
- Fixed various setitem with 1d ndarray that does not have a matching
  length to the indexer ([GH5508](https://github.com/pandas-dev/pandas/issues/5508))  
- Bug in getitem with a MultiIndex and `iloc` ([GH5528](https://github.com/pandas-dev/pandas/issues/5528))  
- Bug in delitem on a Series ([GH5542](https://github.com/pandas-dev/pandas/issues/5542))  
- Bug fix in apply when using custom function and objects are not mutated ([GH5545](https://github.com/pandas-dev/pandas/issues/5545))  
- Bug in selecting from a non-unique index with `loc` ([GH5553](https://github.com/pandas-dev/pandas/issues/5553))  
- Bug in groupby returning non-consistent types when user function returns a `None`, ([GH5592](https://github.com/pandas-dev/pandas/issues/5592))  
- Work around regression in numpy 1.7.0 which erroneously raises IndexError from `ndarray.item` ([GH5666](https://github.com/pandas-dev/pandas/issues/5666))  
- Bug in repeated indexing of object with resultant non-unique index ([GH5678](https://github.com/pandas-dev/pandas/issues/5678))  
- Bug in fillna with Series and a passed series/dict ([GH5703](https://github.com/pandas-dev/pandas/issues/5703))  
- Bug in groupby transform with a datetime-like grouper ([GH5712](https://github.com/pandas-dev/pandas/issues/5712))  
- Bug in MultiIndex selection in PY3 when using certain keys ([GH5725](https://github.com/pandas-dev/pandas/issues/5725))  
- Row-wise concat of differing dtypes failing in certain cases ([GH5754](https://github.com/pandas-dev/pandas/issues/5754))  



<a id='whatsnew-0-13-0-contributors'></a>

## Contributors

A total of 77 people contributed patches to this release.  People with a
“+” by their names contributed a patch for the first time.


- Agustín Herranz +  
- Alex Gaudio +  
- Alex Rothberg +  
- Andreas Klostermann +  
- Andreas Würl +  
- Andy Hayden  
- Ben Alex +  
- Benedikt Sauer +  
- Brad Buran  
- Caleb Epstein +  
- Chang She  
- Christopher Whelan  
- DSM +  
- Dale Jung +  
- Dan Birken  
- David Rasch +  
- Dieter Vandenbussche  
- Gabi Davar +  
- Garrett Drapala  
- Goyo +  
- Greg Reda +  
- Ivan Smirnov +  
- Jack Kelly +  
- Jacob Schaer +  
- Jan Schulz +  
- Jeff Tratner  
- Jeffrey Tratner  
- John McNamara +  
- John W. O’Brien +  
- Joris Van den Bossche  
- Justin Bozonier +  
- Kelsey Jordahl  
- Kevin Stone  
- Kieran O’Mahony  
- Kyle Hausmann +  
- Kyle Kelley +  
- Kyle Meyer  
- Mike Kelly  
- Mortada Mehyar +  
- Nick Foti +  
- Olivier Harris +  
- Ondřej Čertík +  
- PKEuS  
- Phillip Cloud  
- Pierre Haessig +  
- Richard T. Guy +  
- Roman Pekar +  
- Roy Hyunjin Han  
- Skipper Seabold  
- Sten +  
- Thomas A Caswell +  
- Thomas Kluyver  
- Tiago Requeijo +  
- TomAugspurger  
- Trent Hauck  
- Valentin Haenel +  
- Viktor Kerkez +  
- Vincent Arel-Bundock  
- Wes McKinney  
- Wes Turner +  
- Weston Renoud +  
- Yaroslav Halchenko  
- Zach Dwiel +  
- chapman siu +  
- chappers +  
- d10genes +  
- danielballan  
- daydreamt +  
- engstrom +  
- jreback  
- monicaBee +  
- prossahl +  
- rockg +  
- unutbu +  
- westurner +  
- y-p  
- zach powers  